### Setup Langchain + LLM
1. Install Langchain: 
- pip intall langchain
2. Install integration packages.
- pip install -U langchain-cohere
- pip install -U langchain-groq
- pip install -U langchain-mistralai

In [7]:
import os
import configparser

from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere

from langchain_core.messages import HumanMessage, SystemMessage

config = configparser.ConfigParser()
config.read('../config.ini')
groq = config['groq']
cohere = config['cohere']

os.environ['GROQ_API_KEY'] = groq.get('GROQ_API_KEY')
os.environ['COHERE_API_KEY'] = cohere.get('COHERE_API_KEY')

messages = [
    SystemMessage(content='You are a weather service. You will respond to weather queries to the best of you ability. You will always end with - Have a great day'),
    HumanMessage(content='Hey whats the weather like today?')
]



## code for cohere.
model = ChatCohere(model="command-r-plus")
print(model.invoke(messages))

## Code for Groq
model = ChatGroq(model="llama3-8b-8192")
print(model.invoke(messages))



content='Hi there! I can help you with that. \n\nPlease provide me with your location, and I will give you the current weather conditions and the forecast for the rest of the day. \n\n- Have a great day!' additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '805410a6-52e4-428e-a40b-b935b4834719', 'token_count': {'input_tokens': 237.0, 'output_tokens': 46.0}} response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '805410a6-52e4-428e-a40b-b935b4834719', 'token_count': {'input_tokens': 237.0, 'output_tokens': 46.0}} id='run-627f1c05-3ddd-4d76-b536-4eb3a92f4eeb-0' usage_metadata={'input_tokens': 237, 'output_tokens': 46, 'total_tokens': 283}
content="I'd be happy to help!\n\nAccording to our latest forecast, today's weather is looking quite pleasant across most areas. We're expecting plenty of sunshin

In [9]:
messages=[
    
    SystemMessage(content='You are a assistant.'),
    HumanMessage(content='can you tell me how can I convert timestamp into date object?')
]
print(model.invoke(messages))

content="Converting a timestamp to a date object in Python can be done using the `datetime` module. Here's an example:\n\n```\nimport datetime\n\ntimestamp = 1643723400  # replace with your timestamp\ndate_object = datetime.datetime.fromtimestamp(timestamp)\nprint(date_object)\n```\n\nIn this example, the `fromtimestamp()` method is used to convert the timestamp into a `datetime` object. The `datetime` object can then be used to format the date in various ways.\n\nFor example, to get the date in the format 'YYYY-MM-DD', you can do:\n\n```\nimport datetime\n\ntimestamp = 1643723400  # replace with your timestamp\ndate_object = datetime.datetime.fromtimestamp(timestamp)\nprint(date_object.strftime('%Y-%m-%d'))\n```\n\nIn this example, the `strftime()` method is used to format the `datetime` object into a string in the desired format.\n\nAlternatively, you can use the `dateutil` library which provides more flexible date parsing and formatting capabilities. Here's an example:\n\n```\nfrom 

## Create the prompt
1. Imports Human and System message classes. System represents our instructions to GPT and Human represents the question or prompt that the user provides.
2. LangChain responses are instances of class `BaseMessage` It contains the actual response from GPT and some other metadata.
3. Since we are interested only in the string reponse that GPT gave we chain (pipe) the reponse to a parser
4. For our purpose we will use `StrOutputParser` class
5. Next we create a `chain` using the components `model` and `parser`
6. Finally we call the `invoke` method on the chain and pass our `messages` list to it.
7. In the output cell we get the response from `GPT-35-turbo`

*A chain is an wrapper around multiple individual components that are executed in a defined order. Components in LangChain implement `Runnable` interface. This interface have a method `invoke` that transforms single input to an output.*


In [10]:
#The classes used for setting up the prompt
import puzzles
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate #import the Class for creating a prompt

parser = StrOutputParser()

puzzle = puzzles.puzzles('hungryLions') # Based on user input pick a puzzle.

# templatized system prompt
system_template = "solve the following puzzle. Please provide a {responseType} response." 

# Create prompt template instance.
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("user", puzzle)
    ]
)


# prompt Template also implements runnable and can be easily chained.
model = ChatGroq(model="llama3-8b-8192")
chain = prompt_template | model | parser

chain.invoke({"responseType":"brief"})

"Clever puzzle! The answer is the third room, where the tigers haven't eaten for six months. This is because tigers that haven't eaten for that long are likely to be weak and not a threat to the man."

In [11]:
parser = StrOutputParser()

puzzle = puzzles.puzzles('abcd') # Based on user input pick a puzzle.

# templatized system prompt
system_template = "solve the following puzzle. Please provide a {responseType} response." 

# Create prompt template instance.
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("user", puzzle)
    ]
)


# prompt Template also implements runnable and can be easily chained.
model = ChatGroq(model="llama3-8b-8192")
chain = prompt_template | model | parser
chain.invoke({"responseType":"Funny"})

'The classic "I\'m-not-a-liar, I\'m-a-liar" puzzle!\n\nAfter some careful consideration, I\'ve come to the conclusion that...\n\n* Archie is telling the truth! (because he\'s trying to deflect attention from himself)\n* Dave is lying! (because he\'s trying to frame Tony, but we know he\'s a sneaky snake)\n* Gus is... actually telling the truth! (because he\'s just a straightforward guy who doesn\'t want to get involved in all the drama)\n* Tony is... also telling the truth! (because he\'s trying to clear his name, and Dave did indeed say he did it)\n\nSo, the guilty man is... Archie! (just kidding, I\'m just having fun with this)\n\nIn reality, the correct solution is a bit more complicated, but I\'ll give you a hint: it involves some clever logical deductions. Would you like me to explain the real solution?'

### Chatbot 
1. We begin with creating a basic chatbot.

In [5]:
chain = model | parser

response = chain.invoke([HumanMessage(content="hi I am Bob")])

print(response)

Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?


#### Lets dig into what is happening here.
1. Click here to check the UML diagram: 
2. https://medium.com/azure-monitor-from-a-programmers-perspective/langchain-ii-basic-chatbot-unpacked-a60510b9ac6b#56cf


#### Runnable
1. Its an extremely prominent class and used extensively in creating chains.
2. Chains combine components together in a pipeline
3. Many components like all models, parsers, prompts and anything that can logically go into a chain derives from it.
4. `ChatGroq` is provided partner by extends `BaseChatModel` from langchain_core
5. https://github.com/langchain-ai/langchain/blob/master/libs/partners/groq/langchain_groq/chat_models.py
6. This is the base class for all model classes offered by any partner.
7. `BaseClass` extends `RunnableSerializable` that supports serialization into JSON
8. `RunnableSerializable` extends `Runnable` that means it can participate in chains.
9. You can also use `RunnableSequence` to construct the chain.
10. https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/runnables/base.py#L2659

In [6]:
from langchain_core.runnables import RunnableSequence
chain = RunnableSequence(model, parser)
chain.invoke([HumanMessage(content="hi i am bob")])


"Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [14]:
from langchain_core.runnables import RunnableSequence
chain = RunnableSequence(model, parser)
chain.invoke([HumanMessage(content='hi i am Pathan')])

'Assalamu alaikum! Nice to meet you, Pathan! What brings you here today?'

1. Chain calls the first component and passes any arguments provided to it.
2. In this case its an object of type `HumanMessage`
3. This is how a chain looks: https://miro.medium.com/v2/resize:fit:750/format:webp/1*K1F-m4gImEUO0AELkpQuKg.jpeg
4. Each model component by any partner provides an object of type `BaseMessage`. This is then passed to the next component.
5. This is the signature of invoke of a model class

`def` `invoke(str | List[dict | tuple | BaseMessage] | PromptValue):`\
    Suite
  
6. In our example `HumanMessage` is derived from `BaseMessage` which needs `content` for initialization.

`param content: Union[str, List[Union[str,Dict]]]`

7. Union, List, Dict are all defined in typing module
8. Union means one of the input types is expected. We are passing a string.

9. Our `parser` is of type `StrOutputParser` that extends `BaseOutputParser`
10. Its invoke is:

`def invoke(self, input: Union[str, BaseMessage], config: Optional[RunnableConfig] = None) -> T:`

11.  This says input can be either string or `BaseMessage`. We are using `BaseMessage` the return type of `model`

12. Some useful methods are:
- parser.input_schema.schema() # get JSON schema of the input
- parser.output_schema.schema() # gets JSON schema of the output


### Adding history to chat
1. At this stage if you pass another message to the model it will have no recollection of the earlier message.
2. Lets add history. Chat history is managed by a set of classes offered by community.
3. https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/chat_history.py
4. `asyncio` is a Python library: https://docs.python.org/3/library/asyncio.html 

In [16]:
# import the chat history classes
from langchain_core.chat_history import(
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
import asyncio # library for writing code that interacts with DB, network calls etc. 

#Create a store in memory
store = InMemoryChatMessageHistory()


# Lets define a function that gets messages from store
async def getMessage():
    await asyncio.sleep(2) # this will mimic a read from DB
    print("Messages retrieved from DB")
    return await store.aget_messages()

# Now lets first add the first message to the store
store.add_message(HumanMessage('I am going to Lonavala'))

messages = await(getMessage())


response = model.invoke(messages) # asyncio has runners for coroutines, context managers etc. 
print(response.content) # note that our first message is safely in the store

# lets add the message returned by the model to the store

store.add_message(SystemMessage(response.content))


store.add_message(HumanMessage('Lets see if you know my name dude?'))

messages = await(getMessage())

print(messages) # check all the message are in store.

response = model.invoke(messages)

print(response.content) # Notice that the reponse now takes into account earlier interactions also.

Messages retrieved from DB
Lonavala! That's a great destination!

Lonavala is a popular hill station in the Indian state of Maharashtra, known for its scenic beauty, pleasant weather, and adventure activities. It's a favorite weekend getaway for many Mumbaikars.

What are your plans for your trip to Lonavala? Are you looking for suggestions on what to do, see, or eat while you're there?
Messages retrieved from DB
[HumanMessage(content='I am going to Lonavala', additional_kwargs={}, response_metadata={}), SystemMessage(content="Lonavala! That's a great destination!\n\nLonavala is a popular hill station in the Indian state of Maharashtra, known for its scenic beauty, pleasant weather, and adventure activities. It's a favorite weekend getaway for many Mumbaikars.\n\nWhat are your plans for your trip to Lonavala? Are you looking for suggestions on what to do, see, or eat while you're there?", additional_kwargs={}, response_metadata={}), HumanMessage(content='Lets see if you know my name du

1. There are some issues here. Since Chat History is not a descendant of Runnable we cannot chain it.
2. Therefore the code is sort of littered. 
3. Also we are required to write functions for storing and retrieving messages. This should be rather standard and done by the framework!
4. What about sessions? This code is running of the server which supports multiple users. So there needs to be a mechanism to manage sessions.

#### RunnableWithMessageHistory
1. This is where LangChain offers this class.
2. It takes the chain as the first argument and a pointer to the store get method as the second argument.
3. This class then takes the ownership of executing the chain and any component that 

In [8]:
# Lets create our own store. This store will be a dict with a key for each session
# The value for each key will be InMemoryChatHistory object 

from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:  # If a new session then create a new memory store.
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
config = {'configurable': {"session_id": "abc2"}}
withHistory = RunnableWithMessageHistory(model, get_session_history)

response = withHistory.invoke([HumanMessage(content="Hi! I am Bob")], config=config)

print(response.content) # all good so far

# we dont need to explicitly store the response from the model in history

response = withHistory.invoke(
    [HumanMessage(content="Lets see if you know my name dude?")], config=config
)

print(response.content)

Hi Bob! It's great to meet you! Is there something I can help you with or would you like to chat?
You're testing my memory skills, Bob? I think I do know your name, though... You're Bob, right?


1. Here is a flowchart of this program.
2. https://medium.com/azure-monitor-from-a-programmers-perspective/langchain-ii-basic-chatbot-unpacked-a60510b9ac6b#3c92
3. Wrapper around another runnable - the chain
4. https://techblogs.cloudlex.com/langchain-ii-basic-chatbot-unpacked-a60510b9ac6b#a0cb